In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_path = "E:\\DataSets\\keisci\\sensor\\sensor_train\\"

In [3]:
data = pd.read_csv(data_path+"sensor_train.csv")
data.head(2)

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id
0,0,27,0.3,-0.3,0.1,0.6,4.5,8.8,0
1,0,108,0.1,-0.0,-0.4,0.4,4.7,8.4,0


# 根据数据总结出来的结论
+ fragment_id 总共有7300条 
+ 每个fragment_id 对应一个behavior_id
+ 每个fragment_id对应的数据条数： max:61, mean:58, min:50
+ time_point 是每个fragment_id采集的时间点, 每个fragment_id是独立的,范围[0,49999]
+ 其他各列的取值范围 acc_x:(-14.7, 34.5) &ensp;   acc_y:(-14.7, 13.4) &ensp;   acc_z:(-12.6, 28.2) &ensp;   acc_xg:(-19.8, 30.2) &ensp;   acc_yg:(-16.9, 14.4) &ensp;   acc_zg:(-8.9, 38.1)

**goupby("behavior_id"), countdistinct("fragment_id")**

In [4]:
behavior_id_ct = data.pivot_table(index='behavior_id',values='fragment_id',aggfunc=lambda x:len(x.unique())).to_dict()

In [5]:
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

from pyecharts import options as opts
from pyecharts.charts import Bar


bar = Bar()\
.add_xaxis(list(behavior_id_ct["fragment_id"].keys()))\
.add_yaxis("fragment_id 数量", list(behavior_id_ct["fragment_id"].values()))\
.set_global_opts(title_opts=opts.TitleOpts(title="behavioro_id 对应的fragment_id数量 ", subtitle=""))\
.set_series_opts(
    label_opts=opts.LabelOpts(is_show=True),
        markline_opts=opts.MarkLineOpts(
            data=[
                opts.MarkLineItem(type_="min", name="最小值"),
                opts.MarkLineItem(type_="max", name="最大值"),
                opts.MarkLineItem(type_="average", name="平均值"),
            ]
        ),
)

# 需要在第一次渲染时加载javascript文件 


C:\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [6]:
bar.load_javascript()

In [7]:
bar.render_notebook()

**绘制每个列根据时间的变化情况**

In [8]:
import pyecharts.options as opts
from pyecharts.charts import Line
def draw_line(data_dict):
    line = (
    Line()
    .add_xaxis(data_dict["time_point"].values())
    .add_yaxis("acc_x", data_dict["acc_x"].values(), is_smooth=True)
    .add_yaxis("acc_y", data_dict["acc_y"].values(), is_smooth=True)
    .add_yaxis("acc_z", data_dict["acc_z"].values(), is_smooth=True)
    .add_yaxis("acc_xg", data_dict["acc_xg"].values(), is_smooth=True)
    .add_yaxis("acc_yg", data_dict["acc_yg"].values(), is_smooth=True)
    .add_yaxis("acc_zg", data_dict["acc_zg"].values(), is_smooth=True)
    .set_global_opts(title_opts=opts.TitleOpts(title="fragment_id:{}".format(list(data_dict["fragment_id"].values())[0]), subtitle="behavior_id: {}".format(list(data_dict["behavior_id"].values())[0])))
    .render_notebook())
    return line

In [9]:
draw_line(data[data["fragment_id"]==0].to_dict())

C:\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


### 每种类型取5个，保存

In [13]:
behavior_id_frag = data.pivot_table(index='behavior_id',values='fragment_id',aggfunc=lambda x:set(x.unique())).to_dict()

In [23]:
rand_sel_map = {}

In [24]:
from random import sample

In [25]:
for k,v in behavior_id_frag['fragment_id'].items():
    rand_sel_map[k] = sample(list(v), 5)

In [30]:
def draw_line_save(data_dict,fragment_id,behavior_id):
    line = (
    Line(init_opts=opts.InitOpts(page_title="line_{}_{}".format(fragment_id,behavior_id)))
    .add_xaxis(data_dict["time_point"].values())
    .add_yaxis("acc_x", data_dict["acc_x"].values(), is_smooth=True)
    .add_yaxis("acc_y", data_dict["acc_y"].values(), is_smooth=True)
    .add_yaxis("acc_z", data_dict["acc_z"].values(), is_smooth=True)
    .add_yaxis("acc_xg", data_dict["acc_xg"].values(), is_smooth=True)
    .add_yaxis("acc_yg", data_dict["acc_yg"].values(), is_smooth=True)
    .add_yaxis("acc_zg", data_dict["acc_zg"].values(), is_smooth=True)
    .set_global_opts(title_opts=opts.TitleOpts(title="fragment_id:{}".format(list(data_dict["fragment_id"].values())[0]), subtitle="behavior_id: {}".format(list(data_dict["behavior_id"].values())[0])))
    .render(".\\tmp\\line_{}_{}.html".format(fragment_id,behavior_id)))

In [37]:
draw_line_save(data[data["fragment_id"]==0].to_dict(), fragment_id='0', behavior_id='0')

C:\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


In [35]:
import warnings
warnings.filterwarnings("ignore")

In [36]:
rand_sel_map
for behavior_id, fragment_ids in rand_sel_map.items():
    for fragment_id in fragment_ids:
        draw_line_save(data[data["fragment_id"]==fragment_id].to_dict(), fragment_id=fragment_id, behavior_id=behavior_id)

C:\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
C:\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationW